# Isochrone using HERE API

As a GIS analyst, I'm passionate about using data to solve real-world problems. One of the things that frustrates me most is traffic congestion. I've spent countless hours sitting in traffic on my commute to and from work, and I know that many other people share my pain.

So, I decided to use my skills to create an isochrone map that would show me which areas of the city are within a reasonable commute time from my office, using my motorcycle as the transportation mode and avoiding toll roads. Isochrone map is a type of thematic map that shows the areas that can be reached within a certain amount of time or distance from a starting point. They are often used to plan transportation routes or to assess the accessibility of different areas. This makes it a valuable tool for planning, transportation, and urban planning. They can help to identify areas that are well-served by transportation, areas that are underserved by transportation, and areas that are within walking distance of essential services. In this project, I will use it to find out which areas are within 15, 30, and 60 minutes of travel time.

The concept to calculate isochrone maps are something like this: 
- Let's say we have a starting point and its coordinate
- Around the starting points, there are X number of road nodes
- Analyze the route and calculate the duration it takes to reach from the starting point to each node

To make the result closer to my commute, the following parameters are added:
- Transport mode is motorcycle
- Avoid any toll road
- Heavy traffic
- The travel times used are 15 minutes, 30 minutes, and 60 minutes.

Fortunately, there are free isochrone providers that we can use. The closest one I can find that calculate the above parameters is from HERE Maps API. You can view the others and its comparison __[here](https://digital-geography.com/comparing-isochrone-apis-an-insight-into-different-providers/)__. 

## Import libraries

In [1]:
from here.platform import Platform
from here.geopandas_adapter import GeoPandasAdapter
import flexpolyline as fp
import geopandas as gpd
import folium
from folium.plugins import Search
from shapely.geometry import Polygon, mapping
from shapely.ops import transform
import json
import os
import time

## Preparing HERE API Services
HERE API consists of multiple services. We will only use two of them:
- The isoline service to get the isochrone.
- The routing service to calculate the furthest achievable distance.

In [2]:
# Initialize Platform
platform = Platform(adapter=GeoPandasAdapter())

# Check to see if isoline services is available in our account
services = platform.list_services()

for service in services:
    print(f"{service.name} ({service.hrn})")
    
services

# Enable isochrone and routing services
isoline = platform.get_service("hrn:here:service::olp-here:routing-isoline-8")
routing = platform.get_service('hrn:here:service::olp-here:routing-8')

HERE Isoline Routing (hrn:here:service::olp-here:routing-isoline-8)
HERE Routing (hrn:here:service::olp-here:routing-8)


## Getting the isochrone data
To avoid exceeding the daily free limit of HERE API services, we will request the isochrone once and save it in JSON format for later use.

In [3]:
# Function to check whether isochrone data already exist or not. If not, hit the API. The function aims to save on the limited API usage

def get_isochrone_data(isochrone_parameter):
    # Check if isochrone data exist
    if os.path.exists('isochrone_result.json'):
        with open('isochrone_result.json', 'r') as f:
            isochrone_data = json.load(f)
            return isochrone_data
    else:
        # If it doesn't, hit API
        isochrone_data = isoline.get('/isolines', params = isochrone_parameter)
        
        # Save the result to API
        json_string = json.dumps(isochrone_data)
        json_file = open('isochrone_result.json', 'w')
        json_file.write(json_string)
        json_file.close()
        
        # Return the API result
        return isochrone_data

The isochrones will be calculated for Friday at 6:30 PM, the busiest rush hour in local area, and toll roads will be avoided. Since isochrones for motorcycles are not available, cars will be used instead.

In [30]:
# Origin point coordinate
origin_point = (-6.2363373, 106.8010171)

# Create isochrone parameter
params = {
    "origin": str(origin_point[0])+', '+str(origin_point[1]), # Origin coordinate
    "range[type]": "time",
    "range[values]": "900,1800,3600", # Time interval
    "transportMode": "car",
    "optimizeFor": "quality", 
    "avoid[features]": "tollRoad",
    "traffic[mode]": "default", # Incorporate traffic data
    "departureTime": "2023-09-22T18:30:00"
}

In [5]:
# Run the function
isochrone_data = get_isochrone_data(params)

HERE API uses flexible polyline encoding, a lossy compressed representation of a list of coordinate pairs or coordinate triples. To decode ir into coordinate pairs, we'll use the decoding library they provide.

In [6]:
# Decode the flexible polygon
def decode_polygon(data):
    for time_interval in data['isolines']:
        time_interval['decoded_polygon'] = []
        decode = fp.decode(time_interval['polygons'][0]['outer'])
        time_interval['decoded_polygon'].append(decode)
    return data

In [7]:
# Run decoding function
decoded_isochrone = decode_polygon(isochrone_data)

Now that we have decoded coordinates and the isochrone data, we'll convert it to GeoDataFrame, for easier spatial analysis

In [34]:
# Create geodataframes from the decoded isochrone polygon
travel_time = [15, 30, 60]

# Create polygon
polygon_coord_list = []
isochrone_id = ['15 Minutes', '30 Minutes', '60 Minutes']

for i in decoded_isochrone['isolines']:
    polygon_coord_list.append(
        Polygon(i['decoded_polygon'][0])
    )

# Convert polygon to geodataframe
gdf_isochrone = gpd.GeoDataFrame(
    {
        'isochrone_id': isochrone_id,
        'travel_time_mins': travel_time,
        'geometry': polygon_coord_list,
    },
)

# Set the CRS/ datum/ projection of the isochrone data. Since the data is in longitude and latitude, we'll use EPSG 4326 or WGS'84
gdf_isochrone = gdf_isochrone.set_crs(4326, allow_override=True)

# Since our coordinates are in 'latitude, longitude' format, while geopandas expect the 'longitude, latitude', we'll reverse them
gdf_isochrone['geometry'] = gpd.GeoSeries(gdf_isochrone['geometry']).map(lambda polygon: transform(lambda x, y: (y, x), polygon))

# Clip overlapping areas
gdf_isochrone['geometry'][2] = gdf_isochrone['geometry'][2].difference(gdf_isochrone['geometry'][1])
gdf_isochrone['geometry'][1] = gdf_isochrone['geometry'][1].difference(gdf_isochrone['geometry'][0])


In order to make each isochrone polygon doesn't overlap the others, let's remove the overlapping areas

In [9]:
exterior_coords_list = []

for index, row in gdf_isochrone.iterrows():
    ext_coord_row = (row['geometry']).exterior.coords[:-1]
    exterior_coords_list.append(ext_coord_row)

# View sample of exterior coordinates
exterior_coords_list[0][:4]

[(106.77372, -6.234741),
 (106.776123, -6.234741),
 (106.778183, -6.235428),
 (106.779556, -6.236801)]

To make the isochrone more informative, we could also find out what's the furthest distance and average speed of each isochrone duration. This would give us a better understanding of the potential travel times and distances within each isochrone.

In [10]:
# Find the distance from origin point to outermost part of each isochrone duration
travel_distance = []

# For efficient API usage, we will store the travel distance data after API requests to JSON file
# Check if travel distance data exist
if os.path.exists('travel_distance.json'):
     with open('travel_distance.json', 'r') as f:
          travel_distance = json.load(f)

else:
        # If travel distance data doesn't exist, request data via API
        for sublist in exterior_coords_list:
            sublist_travel_distance = []
            for coord_pair in sublist:
                # Wait for one second before making the next request
                time.sleep(1)

                # Make request for the routing API
                response = routing.get(
                        '/routes',
                        params={
                            'transportMode': 'car',
                            'origin': str(origin_point[0]) + ',' + str(origin_point[1]),
                            'destination': str(coord_pair[0]) + ',' + str(coord_pair[1]),
                            'departureTime': '2023-09-22T18:30:00',
                            'avoid[features]': 'tollRoad',
                            'units': 'metric',
                            'return': 'summary',
                            'traffic[mode]': 'default'
                        })

                # Get the length of each route
                route_length = response['routes'][0]['sections'][0]['summary']['length']

                # Add route length to each isochrone duration
                sublist_travel_distance.append(route_length)

            # Add all route length to travel_distance
            travel_distance.append(sublist_travel_distance)
        
            # Save the result to API
            json_string = json.dumps(travel_distance)
            json_file = open('travel_distance.json', 'w')
            json_file.write(json_string)
            json_file.close()

In [11]:
# Find the furthest distance and average speed of each isochrone duration
# Furthest distance
furthest_distance = []

for sublist in travel_distance:
    furthest_distance.append(max(sublist))

furthest_distance

gdf_isochrone['furthest_distance'] = furthest_distance

# Average speed
gdf_isochrone['speed'] = 0
gdf_isochrone['speed'][0] = gdf_isochrone['furthest_distance'][0]*4/1000
gdf_isochrone['speed'][1] = gdf_isochrone['furthest_distance'][1]*2/1000
gdf_isochrone['speed'][2] = gdf_isochrone['furthest_distance'][2]/1000

<ipython-input-11-aac074e9a969>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_isochrone['speed'][0] = gdf_isochrone['furthest_distance'][0]*4/1000


In [12]:
gdf_isochrone.to_file('gdf_isochrone.geojson', driver='GeoJSON')
gdf_isochrone

,isochrone_id,travel_time_mins,geometry,furthest_distance,speed
0,15 Minutes,15,"POLYGON ((106.77372 -6.23474, 106.77612 -6.234...",8901,35.604
1,30 Minutes,30,"POLYGON ((106.74866 -6.21551, 106.75072 -6.214...",19130,38.260
2,60 Minutes,60,"POLYGON ((106.67107 -6.21208, 106.67244 -6.207...",33835,33.835


Now that we have all the data that we want, we can proceed to map the data. We'll use Geopandas to plot it. For the plot, let's make that we hover the polygon, it will be highlighted and shows a tooltip containing the travel time, furthest distance, and average speed

In [13]:
# Create map
map = folium.Map(
    location=origin_point,
    zoom_start=11,
    tiles='OpenStreetMap'
)

# Create marker for origion point
folium.Marker(
    location=origin_point,
    icon=folium.Icon('blue'),
    popup='Origin Point'
).add_to(map)

isochrone_map = gdf_isochrone.explore(
    column= 'isochrone_id',
    tooltip= ['travel_time_mins', 'furthest_distance', 'speed'],
    tooltip_kwds = {'aliases': [ 'Travel Time (mins)', 'Furthest Distance (m)', 'Average Speed (km/h)']},
    cmap= 'YlOrRd',
    m = map,
    legend_kwds = {'caption': 'Travel Time'}
)

map

c:\ProgramData\Anaconda3\envs\geo_env\lib\site-packages\mapclassify\classifiers.py:511: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _fisher_jenks_means(values, classes=5, sort=True):


In [16]:
map.save('isochrone_map.html')